<div style="text-align: center">

<b><h1>  </h1></b>
<h2> Assignment 5 - Link Analysis </h2>
<h2> Modern Information Retrieval Course </h2>
<h3> Dr. Asgari </h3>
<h3> Group Members </h3>
Parsa Mohammadian - 98102284
<br/>
Sara Azarnoush - 98170668
<br/>
Kahbod Aeini - 98101209 
<br/>
<br/>
Sharif University of Technology
<br/>
Computer Engineering Department
<hr/>
</div>

### Introduction

---

### Requirements

---

In [90]:
try:
    from google.colab import drive
    COLAB = True
except:
    COLAB = False
    print('Not in Google Colab')

if COLAB:
  drive.mount('/content/drive')

Not in Google Colab


In [91]:
from IPython.display import clear_output


In [92]:
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install numpy
%pip install simpletransformers

clear_output()


In [93]:
import pandas as pd
import nltk
import string
import functools
import sklearn as sk
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score


In [94]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
clear_output()


### Load Data

First, we load the [sentiment140](https://www.kaggle.com/datasets/kazanova/sentiment140) dataset which is provided by the documentation of the project into pandas dataframe. The dataset contains about 200MB of labelled data. After that, we drop redundant columns and keep only the tweet and the sentiment. The sentiment is encoded as a number from 0(negative) and 4(positive). We also remapped them to 0 and 1. Finally, since the data is to large for normal computer to process, we used fraction of the data in the rest of the notebook.

---


In [95]:
if COLAB:
    PATH_TO_SENTIMENT140_DATASET = 'drive/MyDrive/training.1600000.processed.noemoticon.csv'
else:
    PATH_TO_SENTIMENT140_DATASET = '../datasets/training.1600000.processed.noemoticon.csv'
CSV_COLUMNS = ['target', 'id', 'date', 'flag', 'user', 'text']
TEST_SIZE = 0.2


df = pd.read_csv(PATH_TO_SENTIMENT140_DATASET)
df.columns = CSV_COLUMNS
df.drop(columns=['id', 'date', 'flag', 'user'], inplace=True)
df['target'] = df['target'].map({0: 0, 4:1})
df.head()


,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


#### Sample Data

Limit the size of the dataset in order to limit resource and time usage.


In [96]:
# df = df.sample(n=10)
df = df.head(n=10)


### Preprocess Data

Here we used nltk package and other defined functions to normalize the data as explained in the introduction. 

---

#### Tokenize Text

In [97]:
df['text_tokenized'] = df['text'].apply(lambda x: nltk.word_tokenize(x))


#### Normalize Text

In [98]:
def contains_any_of(token: list, chars: str) -> bool:
    """
    Returns true if the token contains any of the characters in the given list.
    """
    return any(char in token for char in chars)


def remove_punctuation(tokens: list) -> list:
    """
    Removes punctuation from the given tokens.
    """
    return [token for token in tokens if not contains_any_of(token, string.punctuation+"’‘•")]


def remove_stop_words(tokens: list) -> list:
    """
    Removes stop words from the given tokens.
    """
    remove_stop_words.stop_words = set(nltk.corpus.stopwords.words('english'))
    return [token for token in tokens if token not in remove_stop_words.stop_words]


def normalize(tokens):
    """
    Normalizes the tokens of the lyrics.
    """
    normalization_functions = [remove_punctuation, remove_stop_words]
    return functools.reduce(lambda x, f: f(x), normalization_functions, tokens)


df['text_normalized'] = df['text_tokenized'].apply(normalize)


#### Join Text

In [99]:
df['text_preprocessed'] = df['text_normalized'].apply(lambda x: ' '.join(x))


### Name Entity Recognition

---

In [101]:
for idx, sentence in enumerate(df['text_preprocessed']):
    for sent in nltk.sent_tokenize(sentence):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                print(chunk.label(), ' '.join(c[0] for c in chunk))


PERSON School
ORGANIZATION Blah
GPE Kwesidei
GPE Need
GPE LOLTrish
PERSON Yes Rains
